In [1]:
'''

This code is part of the SIPN2 project focused on improving sub-seasonal to seasonal predictions of Arctic Sea Ice. 
If you use this code for a publication or presentation, please cite the reference in the README.md on the
main page (https://github.com/NicWayand/ESIO). 

Questions or comments should be addressed to nicway@uw.edu

Copyright (c) 2018 Nic Wayand

GNU General Public License v3.0


'''

from ecmwfapi import ECMWFDataServer
import os
import sys
import dask
import numpy as np
from esio import download
import datetime
from esio import EsioData as ed
import pandas as pd

import calendar
from datetime import datetime
import os
from ecmwfapi import ECMWFDataServer

'''
Download models with sea ice forecasts within the 2s2 forecast data archive.

https://software.ecmwf.int/wiki/display/S2S/Models

'''

'\nDownload models with sea ice forecasts within the 2s2 forecast data archive.\n\nhttps://software.ecmwf.int/wiki/display/S2S/Models\n\n'

In [19]:
# ecmwf makes a hindcast everytime they make a new forecast
# they label these hindcast by the Model Version Date
# which oddly is not the current date but rather a few weeks ahead
# Then for eace of these versions, they run a forecast for the same mon and day
# from 1999 to one year ago

most_recent = np.datetime64('today')+ np.timedelta64(14,'D')
year_ago= np.datetime64('today') - np.timedelta64(351,'D')
versions = np.arange(year_ago, most_recent)
print(len(versions))
versions = versions[np.is_busday(versions,weekmask = "Mon Thu")]
print(versions)

365
['2018-08-06' '2018-08-09' '2018-08-13' '2018-08-16' '2018-08-20'
 '2018-08-23' '2018-08-27' '2018-08-30' '2018-09-03' '2018-09-06'
 '2018-09-10' '2018-09-13' '2018-09-17' '2018-09-20' '2018-09-24'
 '2018-09-27' '2018-10-01' '2018-10-04' '2018-10-08' '2018-10-11'
 '2018-10-15' '2018-10-18' '2018-10-22' '2018-10-25' '2018-10-29'
 '2018-11-01' '2018-11-05' '2018-11-08' '2018-11-12' '2018-11-15'
 '2018-11-19' '2018-11-22' '2018-11-26' '2018-11-29' '2018-12-03'
 '2018-12-06' '2018-12-10' '2018-12-13' '2018-12-17' '2018-12-20'
 '2018-12-24' '2018-12-27' '2018-12-31' '2019-01-03' '2019-01-07'
 '2019-01-10' '2019-01-14' '2019-01-17' '2019-01-21' '2019-01-24'
 '2019-01-28' '2019-01-31' '2019-02-04' '2019-02-07' '2019-02-11'
 '2019-02-14' '2019-02-18' '2019-02-21' '2019-02-25' '2019-02-28'
 '2019-03-04' '2019-03-07' '2019-03-11' '2019-03-14' '2019-03-18'
 '2019-03-21' '2019-03-25' '2019-03-28' '2019-04-01' '2019-04-04'
 '2019-04-08' '2019-04-11' '2019-04-15' '2019-04-18' '2019-04-22'
 '2019

In [7]:
# test my understanding of date/time
vdates = pd.DatetimeIndex(versions)

#for ct in np.arange(0,len(vdates),1):
for ct in [0]:
    year = vdates[ct].year
    month = vdates[ct].month
    day = vdates[ct].day
    DS = np.datetime64(datetime(1999, month, day, 0, 0))
    DE = np.datetime64(datetime(year-1, month, day, 0, 0))

    hdates = pd.date_range(start=DS,end=DE,freq=pd.DateOffset(years=1))
    hdates = [x.strftime('%Y-%m-%d') for x in hdates]
    print(vdates[ct].strftime('%Y-%m-%d'))
    print(hdates)

2018-08-06
['1999-08-06', '2000-08-06', '2001-08-06', '2002-08-06', '2003-08-06', '2004-08-06', '2005-08-06', '2006-08-06', '2007-08-06', '2008-08-06', '2009-08-06', '2010-08-06', '2011-08-06', '2012-08-06', '2013-08-06', '2014-08-06', '2015-08-06', '2016-08-06', '2017-08-06']


In [9]:
E = ed.EsioData.load()
main_dir = E.model_dir

In [10]:
# a starting point
mod_dicts = {}

# # ECMWF
mod_dicts['ecmwf'] = {
 "class": "s2",
 "dataset": "s2s",
 "expver": "prod",
 "date": "2019-07-11",                                                               
 "hdate": "2005-07-11",                                                          
 "levtype": "sfc",
 "model": "glob",
 "origin": "ecmf",
 "number": "1/2/3/4/5/6/7/8/9/10",
 "param": "31",
 "step": "0-24/24-48/48-72/72-96/96-120/120-144/144-168/168-192/192-216/216-240/240-264/264-288/288-312/312-336/336-360/360-384/384-408/408-432/432-456/456-480/480-504/504-528/528-552/552-576/576-600/600-624/624-648/648-672/672-696/696-720/720-744/744-768/768-792/792-816/816-840/840-864/864-888/888-912/912-936/936-960/960-984/984-1008/1008-1032/1032-1056/1056-1080/1080-1104",
 "stream": "enfh",
 "time": "00:00:00",
 "type": "pf",
 "target": "output",
 }


In [11]:
@dask.delayed  # pairs with the X.compute() command below

def download_month(config_dict):
    # Start server
    cserver = ECMWFDataServer()
    cserver.retrieve(config_dict)
    return 1

In [18]:
dataclass='s2s'
run_type='reforecast'
cmod = 'ecmwf'

vdates = pd.DatetimeIndex(versions)

X = 1
for ct in np.arange(0,len(vdates),1):

    year = vdates[ct].year
    month = vdates[ct].month
    day = vdates[ct].day
    print(year, month, day)
    DS = np.datetime64(datetime(year-20, month, day, 0, 0))
    DE = np.datetime64(datetime(year-1, month, day, 0, 0))

    hdates = pd.date_range(start=DS,end=DE,freq=pd.DateOffset(years=1))
    hdates = [x.strftime('%Y-%m-%d/') for x in hdates]
    vdate = vdates[ct].strftime('%Y-%m-%d')
    
    hdates="".join(hdates)
    hdates=hdates[0:(len(hdates)-1)] # remove last slash
    cdict = mod_dicts[cmod]
    cdict['date'] = vdate
    cdict['hdate'] = hdates
    
    print(hdates)
    
    target = os.path.join(main_dir, cmod, run_type, 'native',
                                  cmod+'_'+hdates[5:7]+'_'+hdates[8:10]+'.grib')

    cdict['target'] = target
    cdict['expect'] = 'any'

    print('output file is ',target)
    X = X + download_month(cdict)

    # print('\n\nNOW REQUESTING DOWNLOAD FOR VERSION',vdate,' HDATE ',hdates)
    # Call compute to download all years concurently for this date, pairs with the dask.delayed command above
X.compute()

2018 9 27
1998-09-27/1999-09-27/2000-09-27/2001-09-27/2002-09-27/2003-09-27/2004-09-27/2005-09-27/2006-09-27/2007-09-27/2008-09-27/2009-09-27/2010-09-27/2011-09-27/2012-09-27/2013-09-27/2014-09-27/2015-09-27/2016-09-27/2017-09-27
output file is  /home/disk/sipn/nicway/data/model/ecmwf/reforecast/native/ecmwf_09_27.grib
2018 10 1
1998-10-01/1999-10-01/2000-10-01/2001-10-01/2002-10-01/2003-10-01/2004-10-01/2005-10-01/2006-10-01/2007-10-01/2008-10-01/2009-10-01/2010-10-01/2011-10-01/2012-10-01/2013-10-01/2014-10-01/2015-10-01/2016-10-01/2017-10-01
output file is  /home/disk/sipn/nicway/data/model/ecmwf/reforecast/native/ecmwf_10_01.grib
2018 10 4
1998-10-04/1999-10-04/2000-10-04/2001-10-04/2002-10-04/2003-10-04/2004-10-04/2005-10-04/2006-10-04/2007-10-04/2008-10-04/2009-10-04/2010-10-04/2011-10-04/2012-10-04/2013-10-04/2014-10-04/2015-10-04/2016-10-04/2017-10-04
output file is  /home/disk/sipn/nicway/data/model/ecmwf/reforecast/native/ecmwf_10_04.grib
2018 10 8
1998-10-08/1999-10-08/2000-

2019-07-21 13:56:14 ECMWF API python library 1.5.4
2019-07-21 13:56:14 ECMWF API at https://api.ecmwf.int/v12019-07-21 13:56:14 ECMWF API python library 1.5.4
2019-07-21 13:56:14 ECMWF API at https://api.ecmwf.int/v1
2019-07-21 13:56:14 ECMWF API python library 1.5.4
2019-07-21 13:56:14 ECMWF API at https://api.ecmwf.int/v1

2019-07-21 13:56:14 ECMWF API python library 1.5.4
2019-07-21 13:56:14 ECMWF API at https://api.ecmwf.int/v1
2019-07-21 13:56:14 ECMWF API python library 1.5.4
2019-07-21 13:56:14 ECMWF API at https://api.ecmwf.int/v1
2019-07-21 13:56:14 ECMWF API python library 1.5.4
2019-07-21 13:56:14 ECMWF API python library 1.5.42019-07-21 13:56:14 ECMWF API python library 1.5.4
2019-07-21 13:56:14 ECMWF API at https://api.ecmwf.int/v1
2019-07-21 13:56:14 ECMWF API python library 1.5.4
2019-07-21 13:56:14 ECMWF API at https://api.ecmwf.int/v1

2019-07-21 13:56:14 ECMWF API at https://api.ecmwf.int/v1
2019-07-21 13:56:14 ECMWF API python library 1.5.4
2019-07-21 13:56:14 ECMWF 

2019-07-21 13:57:22 Request submitted
2019-07-21 13:57:22 Request id: 5d34d1b2bcd319ecbb7adf09
2019-07-21 13:57:22 Request is submitted
2019-07-21 13:57:22 Request submitted
2019-07-21 13:57:22 Request id: 5d34d1b2bbd91cde2b0864d7
2019-07-21 13:57:22 Request is submitted
2019-07-21 13:57:22 Request submitted
2019-07-21 13:57:22 Request id: 5d34d1b2c8ca19f3245c9b17
2019-07-21 13:57:22 Request is submitted
2019-07-21 13:57:22 Request submitted
2019-07-21 13:57:22 Request id: 5d34d1b22fd81cfff248567b
2019-07-21 13:57:22 Request is submitted
2019-07-21 13:57:22 Request submitted
2019-07-21 13:57:22 Request id: 5d34d1b2cad519fa7650bf2a
2019-07-21 13:57:22 Request is submitted
2019-07-21 13:57:23 Request is queued
2019-07-21 13:57:23 Request is queued
2019-07-21 13:57:23 Request is queued
2019-07-21 13:57:23 Request is queued
2019-07-21 13:57:24 Request is queued
Calling 'nice mars /tmp/2c/e9/tmp-_marsk1e6AN.req'
mars - WARN -
mars - WARN - From 29 January 2019 10AM (UTC) MARS uses the inter

mars - INFO   - 20190721.205725 - Transfering 388402800 bytes
mars - INFO   - 20190721.205805 - 9200 fields retrieved from 'marsth'
mars - WARN   - 20190721.205805 - Visiting database marsth : expected 0, got 9200
mars - INFO   - 20190721.205805 - Request time:  wall: 45 sec  cpu: 37 sec
mars - INFO   - 20190721.205805 -   Read from network: 370.41 Mbyte(s) in 6 sec  [60.54 Mbyte/sec]
mars - INFO   - 20190721.205805 -   Processing in marsth: wall: 2 sec
mars - INFO   - 20190721.205805 -   Visiting marsth: wall: 45 sec
mars - INFO   - 20190721.205805 -   Post-processing: wall: 19 sec cpu: 19 sec
mars - INFO   - 20190721.205805 -   Writing to target file: 370.41 Mbyte(s) in < 1 sec [572.26 Mbyte/sec]
mars - INFO   - 20190721.205805 - Memory used: 38.33 Mbyte(s)
mars - INFO   - 20190721.205805 - No errors reported
Process '['nice', 'mars', '/tmp/05/e8/tmp-_marsJBkPpk.req']' finished
2019-07-21 13:58:06 Request is complete
2019-07-21 13:58:06 Transfering 370.41 Mbytes into /home/disk/sipn/

2019-07-21 13:58:12 Request is active
2019-07-21 13:58:16 Welcome Cecilia Bitz
2019-07-21 13:58:17 In case of problems, please check https://confluence.ecmwf.int/display/WEBAPI/Web-API+Troubleshooting or contact servicedesk@ecmwf.int. In case of delays, please check  service activity page https://apps.ecmwf.int/webapi-activity/
mars - INFO   - 20190721.205812 - Server task is 260 [marsth]
mars - INFO   - 20190721.205812 - Request cost: 9,200 fields, 370.41 Mbytes online, nodes: mvr01 [marsth]
mars - INFO   - 20190721.205812 - Transfering 388402800 bytes
2019-07-21 13:58:18 Request submitted
2019-07-21 13:58:18 Request id: 5d34d1eabcd319ece37adf3b
2019-07-21 13:58:18 Request is submitted
Calling 'nice mars /tmp/42/e8/tmp-_marssRWPZi.req'
mars - WARN -
mars - WARN - From 29 January 2019 10AM (UTC) MARS uses the interpolation
mars - WARN - provided by the MIR library. For more details, see
mars - WARN - https://confluence.ecmwf.int/display/UDOC/MARS+interpolation+with+MIR
mars - WARN -
PP

...

mars - INFO   - 20190721.213020 - 9200 fields retrieved from 'marsth'
mars - WARN   - 20190721.213020 - Visiting database marsth : expected 0, got 9200
mars - INFO   - 20190721.213020 - Request time:  wall: 8 min 5 sec  cpu: 43 sec
mars - INFO   - 20190721.213020 -   Read from network: 370.55 Mbyte(s) in < 1 sec [876.29 Mbyte/sec]
mars - INFO   - 20190721.213020 -   Processing in marsth: wall: 7 min 22 sec
mars - INFO   - 20190721.213020 -   Visiting marsth: wall: 8 min 5 sec
mars - INFO   - 20190721.213020 -   Post-processing: wall: 23 sec cpu: 23 sec
mars - INFO   - 20190721.213020 -   Writing to target file: 370.55 Mbyte(s) in < 1 sec [428.35 Mbyte/sec]
mars - INFO   - 20190721.213020 - Memory used: 38.33 Mbyte(s)
mars - INFO   - 20190721.213020 - No errors reported
Process '['nice', 'mars', '/tmp/e6/7d/tmp-_marsgbyyka.req']' finished
2019-07-21 14:30:34 Request is complete
2019-07-21 14:30:34 Transfering 370.554 Mbytes into /home/disk/sipn/nicway/data/model/ecmwf/reforecast/native/

2019-07-21 14:31:28 Transfer rate 3.16673 Mbytes/s
Calling 'nice mars /tmp/86/6d/tmp-_marseqwZ90.req'
mars - WARN -
mars - WARN - From 29 January 2019 10AM (UTC) MARS uses the interpolation
mars - WARN - provided by the MIR library. For more details, see
mars - WARN - https://confluence.ecmwf.int/display/UDOC/MARS+interpolation+with+MIR
mars - WARN -
PPDIR is /var/tmp/ppdir/x86_64
mars - INFO   - 20190721.213059 - Welcome to MARS
mars - INFO   - 20190721.213059 - MARS Client build stamp: 20190610154938
mars - INFO   - 20190721.213059 - MARS Client version: 6.25.3
mars - INFO   - 20190721.213059 - MIR version: 1.2.5
mars - INFO   - 20190721.213059 - Using ecCodes version 2.12.6
mars - INFO   - 20190721.213059 - Using odb_api version: 0.15.10 (file format version: 0.5)
mars - INFO   - 20190721.213059 - Maximum retrieval size is 30.00 G
retrieve,origin=ecmf,hdate=1999-07-18/2000-07-18/2001-07-18/2002-07-18/2003-07-18/2004-07-18/2005-07-18/2006-07-18/2007-07-18/2008-07-18/2009-07-18/2010-0

Calling 'nice mars /tmp/de/b2/tmp-_marsc2Qxvg.req'
mars - WARN -
mars - WARN - From 29 January 2019 10AM (UTC) MARS uses the interpolation
mars - WARN - provided by the MIR library. For more details, see
mars - WARN - https://confluence.ecmwf.int/display/UDOC/MARS+interpolation+with+MIR
mars - WARN -
PPDIR is /var/tmp/ppdir/x86_64
mars - INFO   - 20190721.213142 - Welcome to MARS
mars - INFO   - 20190721.213142 - MARS Client build stamp: 20190610154938
mars - INFO   - 20190721.213142 - MARS Client version: 6.25.3
mars - INFO   - 20190721.213142 - MIR version: 1.2.5
mars - INFO   - 20190721.213142 - Using ecCodes version 2.12.6
mars - INFO   - 20190721.213142 - Using odb_api version: 0.15.10 (file format version: 0.5)
mars - INFO   - 20190721.213142 - Maximum retrieval size is 30.00 G
retrieve,origin=ecmf,hdate=1999-07-25/2000-07-25/2001-07-25/2002-07-25/2003-07-25/2004-07-25/2005-07-25/2006-07-25/2007-07-25/2008-07-25/2009-07-25/2010-07-25/2011-07-25/2012-07-25/2013-07-25/2014-07-25/20

Calling 'nice mars /tmp/09/57/tmp-_mars7vLCbf.req'
mars - WARN -
mars - WARN - From 29 January 2019 10AM (UTC) MARS uses the interpolation
mars - WARN - provided by the MIR library. For more details, see
mars - WARN - https://confluence.ecmwf.int/display/UDOC/MARS+interpolation+with+MIR
mars - WARN -
PPDIR is /var/tmp/ppdir/x86_64
mars - INFO   - 20190721.213218 - Welcome to MARS
mars - INFO   - 20190721.213218 - MARS Client build stamp: 20190610154938
mars - INFO   - 20190721.213218 - MARS Client version: 6.25.3
mars - INFO   - 20190721.213218 - MIR version: 1.2.5
mars - INFO   - 20190721.213218 - Using ecCodes version 2.12.6
mars - INFO   - 20190721.213218 - Using odb_api version: 0.15.10 (file format version: 0.5)
mars - INFO   - 20190721.213218 - Maximum retrieval size is 30.00 G
retrieve,origin=ecmf,hdate=1999-08-01/2000-08-01/2001-08-01/2002-08-01/2003-08-01/2004-08-01/2005-08-01/2006-08-01/2007-08-01/2008-08-01/2009-08-01/2010-08-01/2011-08-01/2012-08-01/2013-08-01/2014-08-01/20

90